In [ ]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:
# ==============================
# Classes
# ==============================
classes = ["incorrect", "correct"]
label_map = {"incorrect":0, "correct":1}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
        text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب {"mask"} ہے۔',
        tokenizer=tokenizer,
)





# templates = [
#     # First 5: mask in the middle
#     ("P1", ManualTemplate(
#         text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب {"mask"} ہے۔',
#         tokenizer=tokenizer
#     )),
#     ("P2", ManualTemplate(
#         text='{"placeholder":"text_a"} کے لیے جواب {"placeholder":"text_b"} صحیح طور پر {"mask"} ہے۔',
#         tokenizer=tokenizer
#     )),
#     ("P3", ManualTemplate(
#         text='مندرجہ ذیل سوال: {"placeholder":"text_a"} انتخاب: {"placeholder":"text_b"} نتیجہ {"mask"} ہونا چاہیے۔',
#         tokenizer=tokenizer
#     )),
#     ("P4", ManualTemplate(
#         text='سوال: {"placeholder":"text_a"} اور آپشن: {"placeholder":"text_b"} درست جواب {"mask"} ہے۔',
#         tokenizer=tokenizer
#     )),
#     ("P5", ManualTemplate(
#         text='{"placeholder":"text_a"} کے سوال کے لیے انتخاب {"placeholder":"text_b"} جواب {"mask"} ہے۔',
#         tokenizer=tokenizer
#     )),

#     # Last 5: mask at the end
#     ("P6", ManualTemplate(
#         text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب ہے {"mask"}',
#         tokenizer=tokenizer
#     )),
#     ("P7", ManualTemplate(
#         text='{"placeholder":"text_a"} کے لیے آپشن: {"placeholder":"text_b"} صحیح جواب {"mask"} ہے۔',
#         tokenizer=tokenizer
#     )),
#     ("P8", ManualTemplate(
#         text='سوال: {"placeholder":"text_a"} انتخاب: {"placeholder":"text_b"} نتیجہ {"mask"}',
#         tokenizer=tokenizer
#     )),
#     ("P9", ManualTemplate(
#         text='{"placeholder":"text_a"} کے سوال کا جواب {"placeholder":"text_b"} نتیجہ {"mask"} ہے۔',
#         tokenizer=tokenizer
#     )),
#     ("P10", ManualTemplate(
#         text='{"placeholder":"text_a"} اور انتخاب: {"placeholder":"text_b"} صحیح جواب {"mask"}',
#         tokenizer=tokenizer
#     )),
# ]


# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
    "correct": ["درست", "صحیح"], 
    "incorrect": ["غلط", "نا درست"]
    },
    tokenizer=tokenizer,
)

In [5]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)


In [6]:
# ==============================
# Load evaluation dataset
# ==============================
csv_path = r"C:\Users\stdFurqan\Desktop\paft\QA\openbookqa_urdu_test_cleaned.csv"
df = pd.read_csv(csv_path)

eval_dataset = []
eval_qids = []   # keeps question ids aligned with examples

for i, row in df.iterrows():
    q = row['urdu_question_stem']
    choices_dict = eval(row['clean_choices'])
    labels = choices_dict['label']
    texts = choices_dict['text']
    answer = row['answerKey']

    for l, t in zip(labels, texts):
        eval_dataset.append(
            InputExample(
                guid=f"eval_{i}_{l}",
                text_a=q,
                text_b=t,
                label=1 if l == answer else 0
            )
        )
        eval_qids.append(i)   # question id


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,   # single template OR loop over templates
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

prompt_model.eval()

question_scores = {}   # qid -> list of (score, label)

with torch.no_grad():
    example_idx = 0

    for batch in eval_loader:
        # batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}

        logits = prompt_model(batch)          # [B, 2]
        pos_scores = logits[:, 1]              # score for "correct" class

        for score, label in zip(pos_scores.cpu(), batch['label'].cpu()):
            qid = eval_qids[example_idx]

            if qid not in question_scores:
                question_scores[qid] = []

            question_scores[qid].append((score.item(), label.item()))
            example_idx += 1


correct = 0
total = 0

for qid, options in question_scores.items():
    # pick option with highest score
    pred_score, pred_label = max(options, key=lambda x: x[0])

    if pred_label == 1:
        correct += 1
    total += 1

qa_accuracy = correct / total
print(f"\n📊 QA Accuracy: {qa_accuracy:.4f}")


tokenizing: 3968it [00:01, 2427.31it/s]



📊 QA Accuracy: 0.2611


In [ ]:
roberta
📊 QA Accuracy: 0.2611

In [ ]:
mbert

📊 QA Accuracy: 0.2460

In [ ]:
# # ==============================
# # Load Evaluation Dataset
# # ==============================
# df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\SST-2\urdu_sentiment_test_labeled.csv")
# eval_dataset = [
#     InputExample(guid=i, text_a=row['text'], label=label_map[row['label']])
#     for i, row in df.iterrows()
# ]

# # ==============================
# # 0-Shot Evaluation with Each Template
# # ==============================
# prompt_model.eval()  # ensure model is in evaluation mode
# batch_size = 8    # eval batch size

# # Optional: store template order and results
# all_pass_patterns = {}

# for pass_idx, (prompt_name, current_template) in enumerate(templates, start=1):
#     print(f"\n🟦 0-Shot Evaluation - Template {prompt_name} ({pass_idx}/{len(templates)})")

#     # Create PromptDataLoader with current template
#     eval_loader = PromptDataLoader(
#         dataset=eval_dataset,
#         tokenizer=tokenizer,
#         template=current_template,
#         tokenizer_wrapper_class=WrapperClass,
#         max_seq_length=128,
#         batch_size=batch_size,
#         shuffle=False
#     )

#     pass_preds = []
#     pass_labels = []

#     # Run evaluation
#     with torch.no_grad():
#         for batch in eval_loader:
#             logits = prompt_model(batch)
#             preds = torch.argmax(logits, dim=-1)
#             pass_preds.extend(preds.cpu().tolist())
#             pass_labels.extend(batch['label'].cpu().tolist())

#     # Print report immediately after this template
#     print(f"\n📊 STS_B Urdu Dev Classification Report - Template {prompt_name}")
#     print(classification_report(pass_labels, pass_preds, target_names=classes, digits=4))

#     # Store template name (optional)
#     all_pass_patterns[f"pass_{pass_idx}"] = prompt_name

# # Optional: print template order at the end
# print("\n✅ Templates used per pass:", all_pass_patterns)
